In [36]:
# Set the PySpark environment variables
import os
import sys
os.environ['SPARK_HOME'] = r"C:\_dev\spark-3.5.1-hadoop3"
# os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
# os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
os.environ['PYSPARK_PYTHON'] = sys.executable

In [37]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc

In [38]:
# Create a SparkSession
spark = SparkSession.builder.appName("DataFrame-Demo").getOrCreate()

### Using RDDs

In [51]:
rdd = spark.sparkContext.textFile(r"D:\pyspark-tutorial\data\data.txt")

# split the words by space
rdd2 = rdd.flatMap(lambda line: line.split(" "))
# ['Apache', 'Spark', 'has', 'its', 'architectural', 'foundation', 'in', 'the', 'resilient', 'distributed',...]
# print(rdd2.collect()) 
# for element in rdd2.collect():
#     print(element)
# 

#for each word, create the tuple (word, 1)
#rdd2.map(lambda word: (word, 1))

# .reduceByKey(lambda a, b: a + b) get an RDD of the count of every unique word by aggregating (adding up) all the 1's you wrote in the last step
 
result_rdd = rdd.flatMap(lambda line: line.split(" ")) \
    .map(lambda word: (word, 1)) \
    .reduceByKey(lambda a, b: a + b) \
    .sortBy(lambda x: x[1], ascending=False)

In [52]:
result_rdd.take(20)

[('the', 12),
 ('of', 7),
 ('a', 7),
 ('distributed', 5),
 ('in', 5),
 ('Spark', 4),
 ('as', 3),
 ('is', 3),
 ('API', 3),
 ('on', 3),
 ('Dataset', 3),
 ('RDD', 3),
 ('and', 2),
 ('results', 2),
 ('its', 2),
 ('data', 2),
 ('cluster', 2),
 ('that', 2),
 ('The', 2),
 ('was', 2)]

### Using DataFrames

In [53]:
df = spark.read.text(r"D:\pyspark-tutorial\data\data.txt")

result_df = df.selectExpr("explode(split(value, ' ')) as word") \
    .groupBy("word").count().orderBy(desc("count"))

In [54]:
result_df.take(20)

[Row(word='the', count=12),
 Row(word='of', count=7),
 Row(word='a', count=7),
 Row(word='in', count=5),
 Row(word='distributed', count=5),
 Row(word='Spark', count=4),
 Row(word='API', count=3),
 Row(word='as', count=3),
 Row(word='RDD', count=3),
 Row(word='is', count=3),
 Row(word='on', count=3),
 Row(word='Dataset', count=3),
 Row(word='data', count=2),
 Row(word='programs', count=2),
 Row(word='its', count=2),
 Row(word='API.', count=2),
 Row(word='and', count=2),
 Row(word='results', count=2),
 Row(word='The', count=2),
 Row(word='function', count=2)]

In [ ]:
spark.stop()